In [ ]:
import pandas as pd
import numpy as np
import scipy.stats
from statsmodels import stats
import statsmodels.stats.multitest as multi
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from scipy.stats import chi2
from scipy.stats import ranksums
from sklearn.metrics import pairwise_distances
import math
import re
from scipy.stats import wilcoxon
from sklearn import linear_model
from scipy.stats import ranksums

In [ ]:
# Cell-type variability analysis
# Distance to the Medoid analysis from Liu et al (2023)
# 
# Code adopted from Liu et al. for Python implementation
# github: https://github.com/jiayiliujiayi/scRNA_Seq-Differential_Variability_Analysis/
# 
# Paper reference: 
# Liu, J., Kreimer, A. & Li, W. V. Differential variability analysis of
# single-cell gene expression data. Brief. Bioinform. 24, (2023).


# With Densne input. broken by cluster and genotype, get distances for each cell from every other cell.
# Find the cell with minimum average distance to other cells, this cell is the medoid.
# return list of distances of every cell from the medoid.
def calc_DM(data):
    #make array of sample vectors and all pairwise distances
    vectors = data[["denSNE_1","denSNE_2"]].to_numpy()
    dist_matrix = pairwise_distances(vectors, metric='euclidean')
    
    # find medoid
    # cell with least distance to all cells
    # each row is a cells distance to other cells
    # so index with minimum sum is medoid
    medoid_in = np.argmin(dist_matrix.sum(axis=0))
    #row of medoid has distance of cells to medoid
    DM = dist_matrix[medoid_in]
    return DM

# Run the full analysis.
# Loop through all the clusters to get distance metrics for each genotype.
# Perform wilcoxon rank sum test to compare distance metrics between genotype.
# Correct for multiple comparison testing by BH FDR correction.
def run_DM(data_d, data_n):
    stats = []
    ps = []
    DM_D_average = []
    DM_N_average = []
    cluster_D_size = []
    cluster_N_size = []
    cluster = []
    stds = []
    for i in range(len(data_d)):
        DM_d = calc_DM(data_d[i])
        DM_n = calc_DM(data_n[i])
        stds.append([np.std(DM_d)/math.sqrt(len(data_d[i])), np.std(DM_n)/math.sqrt(len(data_n[i]))])
        #perform Wilcoxon rank sums test
        stat, p = ranksums(DM_d, DM_n, alternative='two-sided')

        stats.append(stat)
        ps.append(p)
        #DM_D_average.append(np.mean(DM_d))
        #DM_N_average.append(np.mean(DM_n))
        
        DM_D_average.append(np.median(DM_d))
        DM_N_average.append(np.median(DM_n))
        
        cluster_D_size.append(len(data_d[i]))
        cluster_N_size.append(len(data_n[i]))
        cluster.append(i)
    f_results = pd.DataFrame({"Cluster":cluster,
                              "DM_average_Disease":DM_D_average,
                              "DM_average_Normal":DM_N_average,
                              "Cluster_Disease_Size": cluster_D_size,
                              "Cluster_Normal_Size": cluster_N_size,
                              "Stat":stats,
                              "p": ps})
    p_adj = multi.fdrcorrection(ps)
    f_results['Sig?'] = p_adj[0]
    f_results['Adjusted P'] = p_adj[1]
    return f_results, stds

In [ ]:
# load in densne (output from R script) and meta data
dense = pd.read_csv('filtered/all_matrix_data/densne_embeds/dense_ds_scrnaseq_NPCs.csv')
meta = pd.read_csv('filtered/all_matrix_data/meta_data/meta_data_DS_scrnaseq_NPCS_2.csv')
labels = meta['dx']
clusters = meta['cca_clusters']   #choose whichever cluster from R analysis

In [ ]:
#determine indicies of DS and normal cells
down_indicies = np.asarray(labels == 'T21').nonzero()
normal_indicies = np.asarray(labels == 'Euploid').nonzero()

down_indicies, normal_indicies

In [ ]:
#separate data into clusters by condition
num_clus = len(pd.unique(clusters))
data_d = []
data_n = []
for i in range(num_clus):
    cluster_index = np.asarray(clusters == i).nonzero()
    di = np.intersect1d(down_indicies, cluster_index)
    ni = np.intersect1d(normal_indicies, cluster_index)
    clus_d = dense.iloc[di]
    clus_n = dense.iloc[ni]
    data_d.append(clus_d)
    data_n.append(clus_n)


In [ ]:
#run distance to the medoid analysis
results, stds = run_DM(data_d, data_n)
results

In [ ]:
### Gene-Level variance analysis
# Method derived from Brennecke et al. and Osorio et al. 
# MATLAB code from  scGEAToolBox from sc_hvg function was modeled (Cai, 2019)
# for Python implmentation
# Github: https://github.com/jamesjcai/scGEAToolbox
#
# Paper References:
# Brennecke, P. et al. Accounting for technical noise in single-cell RNA-seq experiments. 
# Nat. Methods 10, 1093–1095 (2013).
#
# Osorio, D. et al. Single-Cell Expression Variability Implies Cell Function. Cells 9, (2019).
#
# Cai, James J. 2019. “scGEAToolbox: A Matlab Toolbox for Single-Cell RNA Sequencing Data Analysis.” 
# Bioinformatics (Oxford, England) 36 (6): 1948–49.
###################################################################
# Input of counts matrix
# return genes with at least 5% of cells expressing gene
def filter_low_genes(df):
    size = len(df)
    threshold = int(size * 0.05)
    #print(threshold)
    ind = np.where(df.astype(bool).sum(axis=0) <= threshold)
    return df.drop(df.columns[ind[0]],axis=1,inplace=False)

# loop through both genotypes' data to filter for each cluster
def filter_all(data_d, data_n):
    d = []
    n = []
    for i in range(len(data_d)):
        d.append(filter_low_genes(data_d[i]))
        n.append(filter_low_genes(data_n[i]))
    return d, n

# Calculate the CV2, means for each gene.
# loops through both genotype's dataframes
def calc_metrics_filt(clusterd, clustern):
    cvd=[]
    cvn=[]
    mean_d=[]
    mean_n=[]
    p=[]
    for column in clusterd.columns:
        countsd = np.array(clusterd[column])
        meand = np.mean(countsd)
        cv2d = np.var(countsd) / (meand**2)
        cvd.append(cv2d)
        mean_d.append(meand)
    
    for column in clustern.columns:
        countsn = np.array(clustern[column])
        meann = np.mean(countsn)
        cv2n = np.var(countsn) / (meann**2)
        cvn.append(cv2n)
        mean_n.append(meann)
    
    f_results_d = pd.DataFrame({"Gene":clusterd.columns,
                              "CV2_DS":cvd,
                              "Mean_DS": mean_d})
    f_results_n = pd.DataFrame({"Gene":clustern.columns,
                              "CV2_Nor":cvn,
                              "Mean_Nor": mean_n})
    return f_results_d, f_results_n

# Input of dataframe with calculated metrics following calc_metrics_filt()
# and genotype, where true is DS. 
# Finds mean threshold where only 5% of genes present with CV2 greater than 0.3
# are above this threshold. After filtering using this threshold, fit a linear model
# following eq CV2{exp} = A1/mean + A0
# returns fit and dataframe
def fit_glm(t, ds):
    if ds == True:
        mean_dx = "Mean_DS"
        cvs_dx = "CV2_DS"
    else:
        mean_dx = "Mean_Nor"
        cvs_dx = "CV2_Nor"
    # need to threshold and filter
    t = t[t[mean_dx] != 0]

    ds_hcv = t[t[cvs_dx] > 0.3]

    #select threshold by geting mean such that
    #only 5% of genes with cv>0.3 are above this
    #threshold

    ds_u_thresh = np.percentile(ds_hcv[mean_dx], 95)

    t_filt = t[t[mean_dx] > ds_u_thresh]
 
    reg = linear_model.LinearRegression()
    reg.fit(np.array(1/t_filt[mean_dx]).reshape(-1, 1),np.array(t_filt[cvs_dx]).reshape(-1, 1))
    

    return reg, t


# input of linear models for both genotypes, and dataframes from both genotypes
# Calculates the expected CV2 usiing model. Finds the residual and stores data
# in each respective dataframe
def calc_residuals(reg, regn, t, tn):
    x = np.array(t['Mean_DS']).reshape(-1,1)
    x2 = np.array(tn['Mean_Nor']).reshape(-1,1)
    ypred = reg.predict(1/x)
    ypred2 = regn.predict(1/x2)
    t['CV2_DS_Exp'] = ypred
    tn['CV2_Nor_Exp'] = ypred2
    
    t['DS_resid'] = np.array(t['CV2_DS']).reshape(-1,1) / ypred
    tn['Nor_resid'] = np.array(tn['CV2_Nor']).reshape(-1,1) / ypred2

# Input of datasets for each dataset
# Residual sampling distribution approximates chi-squared distribution
# Correct for multiple comparisons and save data in dataframes.
def chi_distribution(t, tn, dfd, dfn):
    ds_p = 1-chi2.cdf(t["DS_resid"]*dfd, dfd)  #get the probability upper distribution
    no_p = 1-chi2.cdf(tn["Nor_resid"]*dfn, dfn)
    noL_p = chi2.cdf(tn["Nor_resid"]*dfn, dfn) # get the probability of lower distribution
    dsL_p = chi2.cdf(t["DS_resid"]*dfd, dfd)
    
    t['DS_chi_HVG'] = ds_p
    tn['Nor_chi_HVG'] = no_p
    t['DS_chi_sig'] = multi.fdrcorrection(ds_p)[0]
    tn['Nor_chi_sig'] = multi.fdrcorrection(no_p)[0]
    t['DS_chi_fdr'] = multi.fdrcorrection(ds_p)[1]
    tn['Nor_chi_fdr'] = multi.fdrcorrection(no_p)[1]
    
    tn['Nor_chi_LVG'] = noL_p
    t['DS_chi_LVG'] = dsL_p
    t['DS_chi_sig_LVG'] = multi.fdrcorrection(dsL_p)[0]
    tn['Nor_chi_sig_LVG'] = multi.fdrcorrection(noL_p)[0]
    t['DS_chi_fdr_LVG'] = multi.fdrcorrection(dsL_p)[1]
    tn['Nor_chi_fdr_LVG'] = multi.fdrcorrection(noL_p)[1]
    
# Run the full analysis after filtering input datasets
# returns dataframe with gene-level metrics per genotype per cluster
def run_full(down, normal):
    all_results = []
    for i in range(len(down)):
        resd, resn = calc_metrics_filt(down[i], normal[i])
        reg, resd = fit_glm(resd, True)
        regn, resn = fit_glm(resn, False)
        #reg, regn, res = fit_glm(res)
        #print(res)
        calc_residuals(reg,regn,resd, resn)
        
        dfd = len(down[0])-1
        dfn = len(normal[0])-1
        chi_distribution(resd, resn, dfd, dfn)
        
        
        all_results.append((resd,resn))
    return all_results

# Returns significant HVG from dataframe
def sig_results_HVG(results):
    sig_results_d = []
    sig_results_n = []

    for j in range(len(results)):
        resultd = results[j][0]
        resultn = results[j][1]
        sig_results_d.append(resultd[resultd['DS_chi_sig'] == True])
        sig_results_n.append(resultn[resultn['Nor_chi_sig'] == True])
    return sig_results_d, sig_results_n

# Returns signficant LVG from dataframe
def sig_resultsLVG(results):
    sig_results_d = []
    sig_results_n = []

    for j in range(len(results)):
        resultd = results[j][0]
        resultn = results[j][1]
        sig_results_d.append(resultd[resultd['DS_chi_sig_LVG'] == True])
        sig_results_n.append(resultn[resultn['Nor_chi_sig_LVG'] == True])
    return sig_results_d, sig_results_n

In [ ]:
# load in data
normalized_counts_d = pd.read_csv('filtered/all_matrix_data/deseq_counts/Deseq2_norm_DS_scrnaseq.csv')
rows = pd.read_csv('filtered/all_matrix_data/gene_rows/genes_rows_DS_scrnaseq_NPCs.csv')
normalized_counts_d.index = rows['x']
normalized_counts_d = normalized_counts_d.T.iloc[1:,:]
meta = pd.read_csv('filtered/all_matrix_data/meta_data/meta_data_DS_scrnaseq_NPCS_2.csv')
labels = meta['dx']
clusters = meta['cca_clusters']   #choose whichever cluster from R analysis

In [ ]:
#determine indicies of DS and normal cells
down_indicies = np.asarray(labels == 'T21').nonzero()
normal_indicies = np.asarray(labels == 'Euploid').nonzero()

#separate data into clusters by condition
num_clus = len(pd.unique(clusters))
data_d = []
data_n = []
for i in range(num_clus):
    cluster_index = np.asarray(clusters == i).nonzero()
    di = np.intersect1d(down_indicies, cluster_index)
    ni = np.intersect1d(normal_indicies, cluster_index)
    clus_d = normalized_counts_d.iloc[di]
    clus_n = normalized_counts_d.iloc[ni]
    data_d.append(clus_d)
    data_n.append(clus_n)

In [ ]:
# run the analysis
data_df, data_nf = filter_all(data_d,data_n)
ds_results = run_full(data_df, data_nf)
ds_sig_results = sig_results_HVG(ds_results)
ds_sig_results_LVG = sig_resultsLVG(ds_results)

In [ ]:
# Save excel sheets by HVG and LVG separately
with pd.ExcelWriter('DS_HVG_SCT_5.xlsx') as writer:
    for i in range(len(ds_sig_results[0])):
        ds_sig_results[0][i].to_excel(writer, sheet_name="down"+"_cluster_"+str(i))
        ds_sig_results[1][i].to_excel(writer, sheet_name="WT"+"_cluster_"+str(i))
with pd.ExcelWriter('DS_LVG_SCT_5.xlsx') as writer:
    for i in range(len(ds_sig_results_LVG[0])):
        ds_sig_results_LVG[0][i].to_excel(writer, sheet_name="down"+"_cluster_"+str(i))
        ds_sig_results_LVG[1][i].to_excel(writer, sheet_name="WT"+"_cluster_"+str(i))

In [ ]:
# Save excel sheets per cluster with all genes, HVG and LVG
# These files were used for subsequent analysis from gene_level_analyses.R
xls = pd.ExcelFile('DS_HVG_SCT_5.xlsx')
xls2 = pd.ExcelFile('DS_LVG_SCT_5.xlsx')
for x in xls.sheet_names:
    sheet = pd.read_excel(xls, x)
    sheetL = pd.read_excel(xls2, x)

    if len(sheet) == 0:
        continue
    
    outfile = "Variable_gene_lists/DS_scrnaseq_5/"+ x + ".csv"
    #gene.to_csv(outfile, index=False, header=False)
    if "down" in x:
        #print(sheet['DS_resid'].median())
        #cutoff = sheet['DS_resid'].quantile(0.25)
        #sheet = sheet[sheet["DS_resid"] > cutoff]
        sheet["FDR"] = sheet["DS_chi_fdr"]
        sheetL["FDR"] = sheetL["DS_chi_fdr_LVG"]
    else:
        #print(sheet['Nor_resid'].median())
        #cutoff = sheet['Nor_resid'].quantile(0.25)
        #sheet = sheet[sheet["Nor_resid"] > cutoff]
        sheet["FDR"] = sheet["Nor_chi_fdr"]
        sheetL["FDR"] = sheetL["Nor_chi_fdr_LVG"]
    

    pd.concat([sheet, sheetL]).to_csv(outfile, index=False)